# PWMの値の設定

## Boardの判定

In [37]:
import Jetson.GPIO as GPIO

BOARD_NAME=GPIO.gpio_pin_data.get_data()[0]
if BOARD_NAME == "JETSON_NX":
    print("Jetson Xavier NXを認識")
    I2C_BUSNUM = 8
elif BOARD_NAME == "JETSON_XAVIER":
    print("Jetson AGX Xavierを認識")
    I2C_BUSNUM = 8
elif BOARD_NAME == "JETSON_NANO":
    print("Jetson Nanoを認識")
    I2C_BUSNUM = 1
elif BOARD_NAME == "JETSON_ORIN":
    print("Jetson AGX Orinを認識")
    I2C_BUSNUM = 7
elif BOARD_NAME == "JETSON_ORIN_NANO":
    print("Jetson Orin Nanoを認識")
    I2C_BUSNUM = 7

Jetson Orin Nanoを認識


## I2Cの認識確認

0x08: PWMの値の吸い上げ<br>
0x3c: OLED　Display<br>
0x40: PCA9685(PWMの出力)<br>
0x70: PCA9685(PWMの出力)<br>

In [38]:
!i2cdetect -y -r $I2C_BUSNUM

     0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:          -- -- -- -- -- 08 -- -- -- -- -- -- -- 
10: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
20: 20 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
30: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
40: 40 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
50: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
60: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
70: 70 -- -- -- -- -- -- --                         


## ラジコンの制御

Jetson Nano 2G, 4GはBUSNUMを1に指定しています。<br>
Jetson NXは、BUSNUMを8に指定しています。<br>

タミヤのラジコンは60Hzを指定して制御します。<br>

## PWMの値の読み込み

過去に設定されたPWMの値がある場合は読み込みます。

In [39]:
import Fabo_PCA9685
import time
import pkg_resources
import smbus
import time
import json

INITIAL_VALUE = 400

try:
    with open('pwm_params.json') as f:
        json_str = json.load(f)
        print("=================")
        print("Load pwm_params.json")
        print("=================")
        pwm_stop = json_str["pwm_speed"]["stop"]
        pwm_front = json_str["pwm_speed"]["front"]
        pwm_back = json_str["pwm_speed"]["back"]
        pwm_left = json_str["pwm_steering"]["left"]
        pwm_center = json_str["pwm_steering"]["center"]
        pwm_right = json_str["pwm_steering"]["right"]
        steering_width = abs(pwm_center - pwm_left)
        if pwm_right > pwm_left:
            reverse_flag = False
        else:
            reverse_flag = True
        front_width = pwm_front - pwm_stop
        back_width = pwm_back - pwm_stop
        print(f"stop: {pwm_stop}, front: {pwm_front}, back: {pwm_back}")
        print(f"front_width: {front_width}, back_width: {back_width}")
        print(f"left: {pwm_left}, center: {pwm_center}, right: {pwm_right}")
        print(f"steering_width: {steering_width}, reverse_flag: {reverse_flag}")
except Exception as e:
    print(f"Error:{e}")
    print("=================")
    print("New setting")
    print("=================")
    pwm_center = INITIAL_VALUE
    pwm_right = INITIAL_VALUE + 100
    pwm_left = INITIAL_VALUE - 100
    pwm_stop = INITIAL_VALUE
    pwm_back = INITIAL_VALUE
    pwm_front = pwm_stop 
    pwm_back = pwm_back
    front_width = pwm_front - pwm_stop
    back_width = pwm_back - pwm_stop
    steering_width = 100
    reverse_flag = False
    print(f"stop: {pwm_stop}, front: {pwm_front}, back: {pwm_back}")
    print(f"left: {pwm_left}, center: {pwm_center}, right: {pwm_right}")

SMBUS='smbus'
BUSNUM=I2C_BUSNUM
SERVO_HZ=60
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,pwm_stop,address=0x40)
PCA9685.set_hz(SERVO_HZ)

Load pwm_params.json
stop: 410, front: 310, back: 490
front_width: -100, back_width: 80
left: 300, center: 390, right: 480
steering_width: 90, reverse_flag: False


## 保存用の処理とWidgetを作成

In [40]:
import json
import ipywidgets

save_button = ipywidgets.Button(description='値を保存')

def save(change):
    global left_widget, center_widget, right_widget, front_widget, stop_widget, back_widget
    json_str = {
        "pwm_steering": {
            "left": left_widget.value,
            "center": center_widget.value,
            "right": right_widget.value,
        },
        "pwm_speed": {
            "front": front_widget.value,
            "stop": stop_widget.value,
            "back": back_widget.value,
        }
    }

    with open('pwm_params.json', 'w') as f:
        json.dump(json_str, f, ensure_ascii=False)


save_button.on_click(save)

## 値を格納するWidgetを定義

In [41]:
from ipywidgets import Button, Layout, Textarea, HBox, VBox
import ipywidgets

center_widget = ipywidgets.IntText(description='中央', value=pwm_center)
right_widget = ipywidgets.IntText(description='右', value=pwm_right)
left_widget = ipywidgets.IntText(description='左', value=pwm_left)
size_slider = ipywidgets.IntSlider(description='振幅', min=0, max=150, step=1, value=steering_width, orientation='horizontal')
check_button = ipywidgets.Button(description='Check Center')
check_left_button = ipywidgets.Button(description='Check Left(左)')
check_right_button = ipywidgets.Button(description='Check Right(右)')
check_box = ipywidgets.Checkbox(reverse_flag, description='Reserve')

stop_widget = ipywidgets.IntText(description='停止', value=pwm_stop)
check_button = ipywidgets.Button(description='Set Stop')
front_slider = ipywidgets.IntSlider(description='前進幅', min=-100, max=100, step=1, value=front_width, orientation='horizontal')
front_widget = ipywidgets.IntText(description='前進', value=pwm_front)
back_slider = ipywidgets.IntSlider(description='後進幅', min=-100, max=100, step=1, value=back_width, orientation='horizontal')
back_widget = ipywidgets.IntText(description='後進', value=pwm_back)
stop_button = ipywidgets.Button(description='Stop')
set_front_button = ipywidgets.Button(description='Set front')
set_back_button = ipywidgets.Button(description='Set back')

## ハンドルのPWM値の設定

In [42]:
import sys
import threading
import time

STEERING_CH = 0
THROTTLE_CH = 1

pwm_left = 0
pwm_center = 0
pwm_right = 0

reserve = 0

def check_center(c):
    global pwm_center,center_widget
    pwm_center = center_widget.value
    PCA9685.set_channel_value(STEERING_CH, center_widget.value)

def check_left(c):
    global pwm_left,left_widget
    pwm_left = left_widget.value
    PCA9685.set_channel_value(STEERING_CH, pwm_left)
    
def check_right(c):
    global pwm_right,right_widget
    pwm_right = right_widget.value
    PCA9685.set_channel_value(STEERING_CH, pwm_right)
    
def check_left_right(c):
    global pwm_left, pwm_center, pwm_right
    pwm_left = left_widget.value
    PCA9685.set_channel_value(STEERING_CH, pwm_left)
    time.sleep(1)
    pwm_right = right_widget.value
    PCA9685.set_channel_value(STEERING_CH, pwm_right)
    time.sleep(1)
    pwm_center = center_widget.value
    PCA9685.set_channel_value(STEERING_CH, center_widget.value)
    
def on_value_change(change):
    global reserve, pwm_left, pwm_center, pwm_right
    if reserve == 0:
        pwm_left = int(center_widget.value - size_slider.value)
        pwm_right = int(center_widget.value + size_slider.value)
    elif reserve == 1:
        pwm_left = int(center_widget.value + size_slider.value)
        pwm_right = int(center_widget.value - size_slider.value)
    
    left_widget.value = pwm_left
    right_widget.value = pwm_right

def on_reserve(change):
    global reserve, pwm_left, pwm_center, pwm_right
    reserve^=True
    if reserve == 0:
        pwm_left = int(center_widget.value - size_slider.value)
        pwm_right = int(center_widget.value + size_slider.value)
    elif reserve == 1:
        pwm_left = int(center_widget.value + size_slider.value)
        pwm_right = int(center_widget.value - size_slider.value)
    
    left_widget.value = pwm_left
    right_widget.value = pwm_right
    
center_widget.observe(on_value_change)
size_slider.observe(on_value_change)
check_box.observe(on_reserve)
check_button.on_click(check_center)
check_left_button.on_click(check_left)
check_right_button.on_click(check_right)
#left_widget.value = center_widget.value - size_slider.value
#right_widget.value = center_widget.value + size_slider.value

1. ハンドルがまっすぐになる用に中央のPWM値を、設定します。<br>
Check Centerのボタンを押すと実際に値が反映されます。<br>
2. 左右の振幅を調整しながら、Check Left, Check Rightボタンで左右の値を調整します。<br>

In [43]:
check_steering_widget = ipywidgets.VBox([
    ipywidgets.HBox([center_widget, check_button]),
    ipywidgets.HBox([size_slider,check_box]),
    ipywidgets.HBox([left_widget,check_left_button,right_widget,check_right_button]),
    save_button
])

display(check_steering_widget)

## 前進・後進・停止の値の設定

In [44]:
def set_stop(c):
    global stop_widget,pwm_stop,front_slider, back_slider
    pwm_stop = stop_widget.value
    front_widget.value = stop_widget.value + front_slider.value 
    back_widget.value = stop_widget.value + back_slider.value
    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
    time.sleep(0.1)
    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
    
def set_front(change):
    global stop_widget, front_slider, pwm_front
    pwm_front = stop_widget.value + front_slider.value
    PCA9685.set_channel_value(THROTTLE_CH, pwm_front)

def set_back(change):
    global stop_widget, back_slider, pwm_back
    pwm_stop = stop_widget.value
    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
    time.sleep(0.1)
    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
    time.sleep(0.1)
    pwm_back = stop_widget.value + back_slider.value
    PCA9685.set_channel_value(THROTTLE_CH, pwm_back)
    
def on_front_change(change):
    global front_slider, front_widget, stop_widget
    front_widget.value = stop_widget.value + front_slider.value
    
def on_back_change(change):
    global back_slider, back_widget, stop_widget
    back_widget.value = stop_widget.value + back_slider.value
    
def stop(change):
    global stop_widget,speed_slider,pwm_stop
    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
    
front_slider.observe(on_front_change)
back_slider.observe(on_back_change)
check_button.on_click(set_stop)
stop_button.on_click(stop)
set_front_button.on_click(set_front)
set_back_button.on_click(set_back) 

1. 車輪が停止する位置の値を設定し Set Stopボタンで設定します。
2. 前進幅の値を移動して、前進の値を調整します。Set frontボタンで設定します。
前進の値は、停止の値より小さくなる場合もあります(ラジコン仕様に依存)。
3. 後新幅の値を移動して、更新の値を調整し、Set backボタンで設定します。

In [45]:
check_speed_widget = ipywidgets.VBox([
    ipywidgets.HBox([stop_widget, check_button]),
    ipywidgets.HBox([front_slider,front_widget,set_front_button]),
    ipywidgets.HBox([back_slider,back_widget,set_back_button]),
    ipywidgets.HBox([stop_button]),
    save_button
])

display(check_speed_widget)

## 設定された値の確認

設定された値の確認をおこないます。

In [46]:
import json

save_button = ipywidgets.Button(description='Save')

def save(change):
    global left_widget, center_widget, right_widget, front_widget, stop_widget, back_widget
    json_str = {
        "pwm_steering": {
            "left": left_widget.value,
            "center": center_widget.value,
            "right": right_widget.value,
        },
        "pwm_speed": {
            "front": front_widget.value,
            "stop": stop_widget.value,
            "back": back_widget.value,
        }
    }

    with open('pwm_params.json', 'w') as f:
        json.dump(json_str, f, ensure_ascii=False)


save_button.on_click(save)
        
check_value_widget = ipywidgets.VBox([
    ipywidgets.HBox([left_widget, center_widget, right_widget]),
    ipywidgets.HBox([front_widget, stop_widget, back_widget]),
])

display(check_value_widget)

## 動作確認

設定したを実際に設定し、動きを確認します。

In [47]:
with open('pwm_params.json') as f:
    json_str = json.load(f)
    
    front = json_str["pwm_speed"]["front"]
    stop = json_str["pwm_speed"]["stop"]
    back = json_str["pwm_speed"]["back"]
    left = json_str["pwm_steering"]["left"]
    center = json_str["pwm_steering"]["center"]
    right = json_str["pwm_steering"]["right"]

In [93]:
check_left_widget = ipywidgets.IntText(description='左', value=left)
check_center_widget = ipywidgets.IntText(description='中央', value=center)
check_right_widget = ipywidgets.IntText(description='右', value=right)

speed_raw_slider = ipywidgets.IntSlider(description='Speed raw', min=1, max=224, step=1, value=80, orientation='horizontal')

check_front_widget = ipywidgets.IntText(description='前進', value=front)
check_stop_widget = ipywidgets.IntText(description='停止', value=stop)
check_back_widget = ipywidgets.IntText(description='後進', value=back)

run_button = ipywidgets.Button(description='Run')
stop_button = ipywidgets.Button(description='Stop')
check_center_button = ipywidgets.Button(description='Check')
check_left_button = ipywidgets.Button(description='Check')
check_right_button = ipywidgets.Button(description='Check')
check_front_button = ipywidgets.Button(description='Check')
check_stop_button = ipywidgets.Button(description='Check')
check_back_button = ipywidgets.Button(description='Check')

def map_rc(x, in_min, in_max, out_min, out_max):
    return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min
def throttle(speed):
    global pwm_front,pwm_back,THROTTLE_CH,PCA9685
    speed = map_rc(speed, 224, 0, pwm_front, pwm_stop)
    PCA9685.set_channel_value(THROTTLE_CH, speed)
def check_stop(change):
    PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
def check_run(change):
    throttle(speed_raw_slider.value)

    
def check_front(change):
    global check_front_widget
    PCA9685.set_channel_value(THROTTLE_CH, check_front_widget.value)
def check_back(change):
    global check_back_widget,check_stop_widget
    PCA9685.set_channel_value(THROTTLE_CH, check_stop_widget.value)
    time.sleep(0.1)
    PCA9685.set_channel_value(THROTTLE_CH, check_stop_widget.value)
    time.sleep(0.1)
    PCA9685.set_channel_value(THROTTLE_CH, check_back_widget.value)
def check_center(change):
    global check_center_widget
    PCA9685.set_channel_value(STEERING_CH, check_center_widget.value)
def check_left(change):
    global check_left_widget
    PCA9685.set_channel_value(STEERING_CH, check_left_widget.value)
def check_right(change):
    global check_right_widget
    PCA9685.set_channel_value(STEERING_CH, check_right_widget.value)
    
check_stop_button.on_click(check_stop)
check_front_button.on_click(check_front)
check_back_button.on_click(check_back)
check_center_button.on_click(check_center)
check_left_button.on_click(check_left)
check_right_button.on_click(check_right)

run_button.on_click(check_run)
stop_button.on_click(check_stop)

In [94]:
from jetcam.csi_camera import CSICamera
import cv2
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget
import os

In [100]:
camera = CSICamera(capture_device=0,width=224, height=224)
#camera = CSICamera(capture_device=1,width=224, height=224)

camera.running = True

In [101]:
import cv2
import numpy as np

def draw_custom_grid(img):
    height, width = img.shape[:2]

    # 縦方向の真ん中に赤い線を描画
    cv2.line(img, (width // 2, 0), (width // 2, height), color=(0, 0, 255), thickness=2)

    # 横方向に5本の線を描画（中心を基準に）
    colors = [(255, 0, 0), (0, 255, 0), (0, 165, 255), (255, 255, 0), (255, 0, 255)]  # 青、緑、オレンジ、灰色、紫
    num_lines = len(colors)
    spacing = int(width / (num_lines + 1))  # 線間の間隔、+1 は中心線を除くため

    # 中心から等間隔に線を描画
    center_y = height // 2
    for i in range(num_lines):
        offset = ((i - num_lines // 2) * spacing) + (spacing if num_lines % 2 == 0 else 0)
        cv2.line(img, (0, center_y + offset), (width, center_y + offset), color=colors[i], thickness=2)

    return img


In [102]:
# ClickableImageWidgetの初期設定
camera_widget = ClickableImageWidget(width=camera.width*2, height=camera.height*2)
# traitlets.dlinkのtransform関数を更新
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=lambda x: bgr8_to_jpeg(draw_custom_grid(x)))

In [103]:
check_final_widget = ipywidgets.VBox([
    camera_widget,
    ipywidgets.HBox([
        ipywidgets.VBox([check_left_widget,check_left_button]),
        ipywidgets.VBox([check_center_widget,check_center_button]),
        ipywidgets.VBox([check_right_widget,check_right_button])]),
    ipywidgets.HBox([check_front_widget,check_stop_widget,check_back_widget]),
    speed_raw_slider,
    ipywidgets.HBox([run_button,stop_button])
])

display(check_final_widget)

## カメラの開放

In [104]:
camera.running = False
camera.cap.release()